In [115]:
from datasets import load_dataset
from IPython.display import display, Markdown
import re

import ipywidgets as widgets
from IPython.display import Markdown, display, clear_output

In [116]:
def remove_boxed(text):
    pattern = r'\\boxed{(.*)}'
    return re.sub(pattern, r'\1', text)

def fix_align(text):
    pattern = r'\\begin{align\*}(.*?)\\end{align\*}'
    return re.sub(
        pattern, 
        r'$$\\begin{align*}\1\\end{align*}$$', 
        text, flags=re.DOTALL
    )

def convert_latex(text):
    # Convert display math mode
    text = re.sub(r'\\\[\s*(.*?)\s*\\\]', r'$$\1$$', text, flags=re.DOTALL)

    # Convert inline math mode
    text = re.sub(r'\\\((.*?)\\\)', r'$\1$', text, flags=re.DOTALL)

    return text

fix = lambda x: convert_latex(fix_align(remove_boxed(x)))

def diagram_in_output(x):
    return "[asy]" in x["solution"]

In [117]:
math = load_dataset("hendrycks/competition_math")

Found cached dataset competition_math (/home/zhangir/.cache/huggingface/datasets/hendrycks___competition_math/default/1.0.0/52c6a268ae72ef772498d27551a3f682dac50cd8befddd0326d758cb6908b5f0)


  0%|          | 0/2 [00:00<?, ?it/s]

In [118]:
print(math["train"][0].keys())

dict_keys(['problem', 'level', 'type', 'solution'])


In [119]:
data = [{
    "input": x["problem"], 
    "output": fix(x["solution"]),
    "meta": {"level": x["level"], "type": x["type"], "id": 10**8+i}
    }
    for i, x in enumerate(math["train"]) if not diagram_in_output(x)
]

In [120]:
def str_of_row(x):
    return f"---INPUT: {x['input']}\n\n---OUTPUT: {x['output']}"

In [121]:
def display_item(data, index=0):
    clear_output(wait=True)
    item = data[index]
    text_display = Markdown(str_of_row(item))

    # Creating the buttons
    next_button = widgets.Button(description="Next")
    prev_button = widgets.Button(description="Previous")

    # Navigate through the dataset
    def navigate(step):
        nonlocal index
        index = min(max(0, index + step), len(data) - 1)
        display_item(data, index)

    next_button.on_click(lambda b: navigate(1))
    prev_button.on_click(lambda b: navigate(-1))

    # Displaying the components
    button_box = widgets.HBox([prev_button, next_button])
    display(button_box)
    display(text_display)
    display(Markdown(f"ID: {item['meta']['id']}"))
    display(Markdown(f"{index}/{len(data)}"))
    display(Markdown(f"Category: {item['meta']['type']}"))
    if "raw" in item:
        display(item["raw"])

In [124]:
print(data[115]["output"])

Since $f$ is the function that adds two, $f^{-1}$ is the function that subtracts two. Since $g$ is the function that divides by $3,$ $g^{-1}$ is the function that triples. This lets us compute from inside out: $$\begin{array}{rl|l}
&f(g^{-1}(f^{-1}(f^{-1}(g(f(19))))))\\
&\quad=f(g^{-1}(f^{-1}(f^{-1}(g(21)))))&\text{added 2}\\
&\quad=f(g^{-1}(f^{-1}(f^{-1}(7))))&\text{divided by 3}\\
&\quad=f(g^{-1}(f^{-1}(5)))&\text{subtracted 2}\\
&\quad=f(g^{-1}(3))&\text{subtracted 2}\\
&\quad=f(9)&\text{tripled}\\
&\quad=11}&\text{added 2\\
\end{array}$$


In [ ]:
display_item(data, index=102)

---INPUT: Given that the graphs of $y=h(x)$ and $y=j(x)$ intersect at $(2,2),$ $(4,6),$ $(6,12),$ and $(8,12),$ there is one point where the graphs of $y=h(2x)$ and $y=2j(x)$ must intersect. What is the sum of the coordinates of that point?

---OUTPUT: The given information tells us that $$\begin{array}{c@{\qquad}c}
h(2)=j(2)=2, & h(4)=j(4)=6, \\
h(6)=j(6)=12, & h(8)=j(8)=12.
\end{array}$$If the graphs of $y=h(2x)$ and $y=2j(x)$ intersect at $(a,b),$ then $$h(2a)=2j(a)= b.$$Checking the possibilities in the table above, we see that $h(8)=2j(4)=12.$ Thus, the graphs of $y=h(2x)$ and $y=2j(x)$ intersect at $(4,12),$ the sum of whose coordinates is $16.$

ID: 100000124

122/6719

Category: Algebra